In [1]:
import re #for regular expressions to allow sorting the frames

def numberRegex(e):
  #Finding all digits in the filename
  x = re.findall("(\d)", e)

  #if there are two digits in filename
  if(len(x)==2):
      return int(x[0]+""+x[1])
  #if there is one digit in the filename
  else:
      return int(x[0])


In [13]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd

#Lucas directory
#C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Ian directory
#H:/Documents/Github repositories/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Directory containing the RF scan data
dataFolder = 'H:/Documents/Github repositories/SE4450-project-code/Preprocessing/P82-W0-S2.mat'
#vecRFDataNames = os.listdir(os.path.join(dataFolder)) #vector with the names of the images in the above folder
#vecRFDataNames.sort(key=numberRegex) #Sorts the images by frame number

#Loading in the matlab file information
rawMatFile = loadmat(dataFolder)

#Contains just the RF data pulled from the .mat file
RFData = [[element for element in upperElement] for upperElement in rawMatFile['rf1']]

zeroArray = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(10):
    RFData[i][0][0] = 0

#rawMatFile['rf1'] = RFData

matDic = {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Oct 21 17:26:33 2020','__version__':'1.0', '__globals__':'','rf1':RFData}

savemat("transformTest.mat", matDic, long_field_names=True)

#Converting python dictionary into pandas dataframe for easy manipulation (unsure if needed)
RFDataframe = pd.DataFrame(RFData)

print(RFDataframe.head(5))
        
#print(RFDataframe.sample(5))

                                                   0  ...                                                255
0  [0, 10, 44, 65, 25, -8, -78, -33, 27, -46, -88...  ...  [-14, 11, 34, 70, 12, 9, -50, -42, 112, -18, -...
1  [0, 0, 31, 62, 22, 11, -55, -55, -4, 32, -115,...  ...  [-10, -1, 17, 56, 38, 42, -25, -27, 119, 55, -...
2  [0, 2, 14, 41, 21, 15, -23, -52, -27, 117, -68...  ...  [61, -26, 5, 36, 50, 51, 20, -21, 74, 117, -39...
3  [0, 0, 2, 32, 36, 28, 0, -30, -52, 131, 1, -10...  ...  [87, -32, -3, 29, 52, 83, 52, -7, 8, 119, 17, ...
4  [0, 22, 3, 23, 31, 25, 17, -28, -47, 110, 91, ...  ...  [60, 62, -20, 26, 39, 68, 82, 16, -44, 86, 93,...

[5 rows x 256 columns]


In [4]:
#This was a long shot to visualize the RF dataframe
import matplotlib.pyplot as plt
import numpy as np

[m,n] = np.shape(RFData[0:10][0:10])

#Colour Map using Matrix
plt.figure()
plt.imshow(RFData, alpha=0.8)
plt.xticks(np.arange(n))
plt.yticks(np.arange(m))
plt.xlabel('Numbers')
plt.ylabel('Value')
plt.title('Color Maps')

Text(0.5, 1.0, 'Color Maps')